<a href="https://colab.research.google.com/github/zyq823/ML_Facial_Recognition/blob/master/facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import math
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout


# initialization for Question 2

def parse_dataset(filename):
    data_file = open(filename, 'r')  # Open File "to read"
    
    images = []
    labels = []

    for i, image in enumerate(data_file):
        if i == 0:
            continue # ignore headers
        emotion, pixelStr = image.strip().split(',')  # strip() removes '\n', and split(',') splits the line at tabs
        pixel_list = pixelStr.split()
        map_pixel = map(float, pixel_list)
        pixels = list(map_pixel)
        
        labels.append(int(emotion)) 
        images.append(pixels) 
    image1D = np.array(images)
    label1D = np.array(labels)
    image3D = image1D.reshape(len(labels), 48, 48)
        
    print("Number of samples by emotion: Angry - {0} , Disgust - {1} , Fear - {2} , Happy - {3} , Sad - {4} , Surprise - {5} , Neutral - {6}".format(len([i for i in label1D if i == 0]), len([i for i in label1D if i == 1]), len([i for i in label1D if i == 2]), len([i for i in label1D if i == 3]), len([i for i in label1D if i == 4]), len([i for i in label1D if i == 5]), len([i for i in label1D if i == 6])))
    return image3D, label1D

train_images, train_labels = parse_dataset('/Q2_Train_Data.csv')
valid_images, valid_labels = parse_dataset('/Q2_Validation_Data.csv')
test_images, test_labels = parse_dataset('/Q2_Test_Data.csv')

print("Train Image Shape: ", train_images.shape, "Train Label Shape: ", train_labels.shape) 
print("Validation Image Shape: ", valid_images.shape, "Validation Label Shape: ", valid_labels.shape) 
print("Test Image Shape: ", test_images.shape, "Test Label Shape: ", test_labels.shape) 

# print("Visualizing 28th image for Anger. ")
# print("Visualizing 300th image for Disgust. ")
# print("Visualizing 3rd image for Fear. ")
# print("Visualizing 8th image for Happy. ")
# print("Visualizing 20th image for Sad. ")
# print("Visualizing 16th image for Surprise. ")
# print("Visualizing 14th image for Neutral. ")
# _ = plt.imshow(train_images[13])
# print(type(train_images))
# print(train_labels)
# print(train_images[2,:,:])
# plt.show()


# Normalize the images
def preprocessing(dataset):
    for image in dataset:
        mean = np.mean(image)
        stddev = np.std(image)
        adjusted_stddev = max(stddev, 1.0/math.sqrt(2304))
        for p, row in enumerate(image):
            for q, col in enumerate(row):
                 image[p][q] = (col - mean) / adjusted_stddev

preprocessing(train_images)
preprocessing(valid_images)
preprocessing(test_images)


Number of samples by emotion: Angry - 3995 , Disgust - 436 , Fear - 4097 , Happy - 7215 , Sad - 4830 , Surprise - 3171 , Neutral - 4965
Number of samples by emotion: Angry - 467 , Disgust - 56 , Fear - 496 , Happy - 895 , Sad - 653 , Surprise - 415 , Neutral - 607
Number of samples by emotion: Angry - 491 , Disgust - 55 , Fear - 528 , Happy - 879 , Sad - 594 , Surprise - 416 , Neutral - 626
Train Image Shape:  (28709, 48, 48) Train Label Shape:  (28709,)
Validation Image Shape:  (3589, 48, 48) Validation Label Shape:  (3589,)
Test Image Shape:  (3589, 48, 48) Test Label Shape:  (3589,)


In [3]:
# Build FNN models
model0 = Sequential([
    Dense(2304, activation='relu', input_shape=(48*48,), name="first_hidden_layer"),
    Dense(2304//2, activation='relu', name="second_hidden_layer"), Dropout(0.25),
    Dense(7, activation='softmax'),
])

model1 = Sequential([ # more layers
    Dense(2304, activation='relu', input_shape=(48*48,), name="first_hidden_layer"),
    Dense(2304//2, activation='relu', name="second_hidden_layer"), Dropout(0.25),
    Dense(2304//2, activation='relu', name="third_hidden_layer"), Dropout(0.25),
    Dense(7, activation='softmax'),
])

model2 = Sequential([ # more layers
    Dense(2304, activation='relu', input_shape=(48*48,), name="first_hidden_layer"),
    Dense(2304//8, activation='relu', name="second_hidden_layer"), Dropout(0.25),
    Dense(2304//8, activation='relu', name="third_hidden_layer"), Dropout(0.25),
    Dense(7, activation='softmax'),
])

model3 = Sequential([ # lower dropout
    Dense(2304, activation='relu', input_shape=(48*48,), name="first_hidden_layer"),
    Dense(2304//2, activation='relu', name="second_hidden_layer"), Dropout(0.20),
    Dense(2304//2, activation='relu', name="third_hidden_layer"), Dropout(0.20),
    Dense(7, activation='softmax'),
])

model4 = Sequential([ # different activation
    Dense(2304, activation='tanh', input_shape=(48*48,), name="first_hidden_layer"),
    Dense(2304//2, activation='tanh', name="second_hidden_layer"), Dropout(0.20),
    Dense(2304//2, activation='tanh', name="third_hidden_layer"), Dropout(0.20),
    Dense(7, activation='softmax'),
])

print(model0.summary())
print(model1.summary())
print(model2.summary())
print(model3.summary())
print(model4.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_hidden_layer (Dense)   (None, 2304)              5310720   
_________________________________________________________________
second_hidden_layer (Dense)  (None, 1152)              2655360   
_________________________________________________________________
dropout (Dropout)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 8071      
Total params: 7,974,151
Trainable params: 7,974,151
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_hidden_layer (Dense)   (None, 2304)              5310720   
___________________

In [12]:
model0.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

flatten_train_images = train_images.reshape((-1, 48*48))
flatten_valid_images = valid_images.reshape((-1, 48*48))
print(train_labels)

model0.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)

[0 0 2 ... 4 0 4]
Epoch 1/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1321 - accuracy: 0.9821
Epoch 2/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1274 - accuracy: 0.9837
Epoch 3/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1274 - accuracy: 0.9832
Epoch 4/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1216 - accuracy: 0.9827
Epoch 5/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1237 - accuracy: 0.9838
Epoch 6/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1140 - accuracy: 0.9865
Epoch 7/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1081 - accuracy: 0.9878
Epoch 8/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1072 - accuracy: 0.9864
Epoch 9/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1024 - accuracy: 0.9883
Epoch 10/10
113/113 [==============================] - 1s 5ms/step - loss: 0.1021 

In [ ]:
performance0 = model0.evaluate(flatten_valid_images, to_categorical(valid_labels))
print("Accuracy on Validation samples: {0}".format(performance0[1]))

113/113 [==============================] - 0s 2ms/step - loss: 2.0586 - accuracy: 0.4558
Accuracy on Validation samples: 0.45583727955818176


In [ ]:
model1.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

model1.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0911 - accuracy: 0.9789
Epoch 2/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0867 - accuracy: 0.9797
Epoch 3/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0740 - accuracy: 0.9852
Epoch 4/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0745 - accuracy: 0.9839
Epoch 5/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0708 - accuracy: 0.9854
Epoch 6/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0672 - accuracy: 0.9862
Epoch 7/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0639 - accuracy: 0.9869
Epoch 8/10
113/113 [==============================] - 1s 7ms/step - loss: 0.0631 - accuracy: 0.9868
Epoch 9/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0612 - accuracy: 0.9872
Epoch 10/10
113/113 [==============================] - 1s 8ms/step - loss: 0.0582 - accuracy: 0.9879

In [ ]:
performance1 = model1.evaluate(flatten_valid_images, to_categorical(valid_labels))
print("Accuracy on Validation samples: {0}".format(performance1[1]))

113/113 [==============================] - 0s 3ms/step - loss: 2.7568 - accuracy: 0.4528
Accuracy on Validation samples: 0.4527723491191864


In [ ]:
model2.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

model2.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0857 - accuracy: 0.9794
Epoch 2/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1632 - accuracy: 0.9629
Epoch 3/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1216 - accuracy: 0.9694
Epoch 4/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0876 - accuracy: 0.9786
Epoch 5/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0820 - accuracy: 0.9797
Epoch 6/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0804 - accuracy: 0.9817
Epoch 7/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0782 - accuracy: 0.9806
Epoch 8/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0735 - accuracy: 0.9828
Epoch 9/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0728 - accuracy: 0.9821
Epoch 10/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0776 - accuracy: 0.9804

In [ ]:
performance2 = model2.evaluate(flatten_valid_images, to_categorical(valid_labels))
print("Accuracy on Validation samples: {0}".format(performance2[1]))

113/113 [==============================] - 0s 3ms/step - loss: 3.1521 - accuracy: 0.4043
Accuracy on Validation samples: 0.4042908847332001


In [24]:
model3.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

model3.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 1s 7ms/step - loss: 0.0998 - accuracy: 0.9793
Epoch 2/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0940 - accuracy: 0.9811
Epoch 3/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0935 - accuracy: 0.9788
Epoch 4/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0838 - accuracy: 0.9830
Epoch 5/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1400 - accuracy: 0.9726
Epoch 6/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0797 - accuracy: 0.9836
Epoch 7/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0772 - accuracy: 0.9846
Epoch 8/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1420 - accuracy: 0.9736
Epoch 9/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0723 - accuracy: 0.9870
Epoch 10/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0677 - accuracy: 0.9873

In [ ]:
performance3 = model3.evaluate(flatten_valid_images, to_categorical(valid_labels))
print("Accuracy on Validation samples: {0}".format(performance3[1]))

113/113 [==============================] - 0s 3ms/step - loss: 2.6530 - accuracy: 0.4405
Accuracy on Validation samples: 0.44051268696784973


In [34]:
model4.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

model4.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 1s 7ms/step - loss: 0.0757 - accuracy: 0.9850
Epoch 2/10
113/113 [==============================] - 1s 6ms/step - loss: 0.4057 - accuracy: 0.9214
Epoch 3/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0704 - accuracy: 0.9859
Epoch 4/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0601 - accuracy: 0.9898
Epoch 5/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0527 - accuracy: 0.9917
Epoch 6/10
113/113 [==============================] - 1s 6ms/step - loss: 0.1935 - accuracy: 0.9647
Epoch 7/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0531 - accuracy: 0.9909
Epoch 8/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0488 - accuracy: 0.9918
Epoch 9/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0606 - accuracy: 0.9887
Epoch 10/10
113/113 [==============================] - 1s 6ms/step - loss: 0.0448 - accuracy: 0.9927

In [ ]:
performance4 = model4.evaluate(flatten_valid_images, to_categorical(valid_labels))
print("Accuracy on Validation samples: {0}".format(performance4[1]))

113/113 [==============================] - 0s 3ms/step - loss: 3.4006 - accuracy: 0.4377
Accuracy on Validation samples: 0.43772637844085693


In [13]:
flatten_test_images = test_images.reshape((-1, 48*48))
performance5 = model0.evaluate(flatten_test_images, to_categorical(test_labels))
print("Accuracy on Test samples: {0}".format(performance5[1]))

113/113 [==============================] - 0s 3ms/step - loss: 2.0240 - accuracy: 0.4494
Accuracy on Test samples: 0.4494287967681885


In [52]:
from keras.layers import Conv2D, Flatten, MaxPooling2D

common_features0 = [Conv2D(32, kernel_size=3, activation='relu', input_shape=(48,48,1)), 
            Conv2D(32, kernel_size=3, activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(64, kernel_size=3, activation='relu'),
            Conv2D(64, kernel_size=3, activation='relu'),
            MaxPooling2D(pool_size=(2,2)), Flatten(),]
classifier0 = [Dense(512, activation='relu'), Dense(7, activation='softmax'),]

common_features1 = [Conv2D(64, kernel_size=3, activation='relu', input_shape=(48,48,1)), 
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(128, kernel_size=3, activation='relu'),
            MaxPooling2D(pool_size=(2,2)), Flatten(),]
classifier1 = [Dense(512, activation='relu'), Dense(7, activation='softmax'),]

common_features2 = [Conv2D(32, kernel_size=2, strides=(2,2), activation='relu', input_shape=(48,48,1)), 
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(64, kernel_size=2, strides=(2,2), activation='relu'),
            MaxPooling2D(pool_size=(2,2)), Flatten(),]
classifier2 = [Dense(512, activation='relu'), Dense(7, activation='softmax'),]

common_features3 = [Conv2D(64, kernel_size=2, activation='relu', input_shape=(48,48,1)), 
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(128, kernel_size=2, activation='relu'),
            MaxPooling2D(pool_size=(2,2)), Flatten(),]
classifier3 = [Dense(512, activation='relu'), Dense(7, activation='softmax'),]

cnn_model0 = Sequential(common_features0+classifier0)
cnn_model1 = Sequential(common_features1+classifier1)
cnn_model2 = Sequential(common_features2+classifier2)
cnn_model3 = Sequential(common_features3+classifier3)

print(cnn_model0.summary())  # Compare number of parameteres against FFN
print(cnn_model1.summary())  # Compare number of parameteres against FFN
print(cnn_model2.summary())  # Compare number of parameteres against FFN
print(cnn_model3.summary())  # Compare number of parameteres against FFN

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_120 (Conv2D)          (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 44, 44, 32)        9248      
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 20, 20, 64)        18496     
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_93 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 5184)            

In [85]:
cnn_model0.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

train_images_3d = train_images.reshape(28709,48,48,1)
valid_images_3d = valid_images.reshape(3589,48,48,1)
test_images_3d = test_images.reshape(3589,48,48,1)

cnn_model0.fit(train_images_3d, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 3s 31ms/step - loss: 0.1844 - accuracy: 0.9669
Epoch 2/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0301 - accuracy: 0.9958
Epoch 3/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0238 - accuracy: 0.9961
Epoch 4/10
113/113 [==============================] - 3s 30ms/step - loss: 0.3407 - accuracy: 0.9331
Epoch 5/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0284 - accuracy: 0.9966
Epoch 6/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0233 - accuracy: 0.9962
Epoch 7/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0232 - accuracy: 0.9961
Epoch 8/10
113/113 [==============================] - 3s 30ms/step - loss: 0.0216 - accuracy: 0.9962
Epoch 9/10
113/113 [==============================] - 3s 31ms/step - loss: 0.0200 - accuracy: 0.9964
Epoch 10/10
113/113 [==============================] - 3s 31ms/step - loss: 0.0201 - accura

In [86]:
performance0 = cnn_model0.evaluate(valid_images_3d, to_categorical(valid_labels))

print("Accuracy on Validation samples: {0}".format(performance0[1]))

113/113 [==============================] - 0s 4ms/step - loss: 3.2723 - accuracy: 0.5266
Accuracy on Validation samples: 0.5266090631484985


In [87]:
cnn_model1.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

cnn_model1.fit(train_images_3d, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0375 - accuracy: 0.9963
Epoch 2/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0342 - accuracy: 0.9966
Epoch 3/10
113/113 [==============================] - 3s 29ms/step - loss: 0.5001 - accuracy: 0.9364
Epoch 4/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0583 - accuracy: 0.9953
Epoch 5/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0422 - accuracy: 0.9966
Epoch 6/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0355 - accuracy: 0.9965
Epoch 7/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0316 - accuracy: 0.9969
Epoch 8/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0298 - accuracy: 0.9964
Epoch 9/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0286 - accuracy: 0.9966
Epoch 10/10
113/113 [==============================] - 3s 29ms/step - loss: 0.0284 - accura

In [88]:
performance1 = cnn_model1.evaluate(valid_images_3d, to_categorical(valid_labels))

print("Accuracy on Validation samples: {0}".format(performance1[1]))

113/113 [==============================] - 0s 3ms/step - loss: 2.3299 - accuracy: 0.5405
Accuracy on Validation samples: 0.5405405163764954


In [102]:
cnn_model2.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

cnn_model2.fit(train_images_3d, to_categorical(train_labels), epochs=10, batch_size=1024,)

Epoch 1/10
29/29 [==============================] - 0s 13ms/step - loss: 1.3070 - accuracy: 0.5109
Epoch 2/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2958 - accuracy: 0.5163
Epoch 3/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2862 - accuracy: 0.5201
Epoch 4/10
29/29 [==============================] - 0s 13ms/step - loss: 1.3126 - accuracy: 0.5078
Epoch 5/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2889 - accuracy: 0.5209
Epoch 6/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2889 - accuracy: 0.5181
Epoch 7/10
29/29 [==============================] - 0s 13ms/step - loss: 1.3112 - accuracy: 0.5065
Epoch 8/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2866 - accuracy: 0.5181
Epoch 9/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2762 - accuracy: 0.5241
Epoch 10/10
29/29 [==============================] - 0s 13ms/step - loss: 1.2835 - accuracy: 0.5200


In [104]:
performance2 = cnn_model2.evaluate(valid_images_3d, to_categorical(valid_labels))

print("Accuracy on Validation samples: {0}".format(performance2[1]))

113/113 [==============================] - 0s 2ms/step - loss: 1.9581 - accuracy: 0.3664
Accuracy on Validation samples: 0.36639732122421265


In [113]:
cnn_model3.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

cnn_model3.fit(train_images_3d, to_categorical(train_labels), epochs=10, batch_size=256,)

Epoch 1/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0501 - accuracy: 0.9940
Epoch 2/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0396 - accuracy: 0.9964
Epoch 3/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0369 - accuracy: 0.9964
Epoch 4/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0344 - accuracy: 0.9967
Epoch 5/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0329 - accuracy: 0.9966
Epoch 6/10
113/113 [==============================] - 3s 26ms/step - loss: 0.3415 - accuracy: 0.9527
Epoch 7/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0455 - accuracy: 0.9964
Epoch 8/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0373 - accuracy: 0.9969
Epoch 9/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0336 - accuracy: 0.9968
Epoch 10/10
113/113 [==============================] - 3s 26ms/step - loss: 0.0310 - accura

In [114]:
performance3 = cnn_model3.evaluate(valid_images_3d, to_categorical(valid_labels))

print("Accuracy on Validation samples: {0}".format(performance3[1]))

113/113 [==============================] - 0s 3ms/step - loss: 2.2212 - accuracy: 0.5297
Accuracy on Validation samples: 0.5296739935874939


In [109]:
performance4 = cnn_model1.evaluate(test_images_3d, to_categorical(test_labels))

print("Accuracy on Test samples: {0}".format(performance4[1]))

113/113 [==============================] - 0s 4ms/step - loss: 2.1901 - accuracy: 0.5428
Accuracy on Test samples: 0.5427695512771606


In [116]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

def optimize_cnn(hyperparameter):
  
  # Define model using hyperparameters 
  cnn_model4 = Sequential([Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], activation='relu', input_shape=(48,48,1)), 
            Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'), 
            MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'), 
            MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']), 
            Flatten(),
            Dense(512, activation='relu'), 
            Dense(7, activation='softmax'),])
  
  cnn_model4.compile(optimizer=hyperparameter['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'],)

  # create a training (23924 samples) and validation (4785 samples) subsets from training images.
  # Validation subset will be used to find the optimal hyperparameters
  train_X, train_y = train_images_3d[:23924], train_labels[:23924]
  valid_X, valid_y = train_images_3d[23924:], train_labels[23924:]

  _ = cnn_model4.fit(train_X, to_categorical(train_y), epochs=2, batch_size=256, verbose=0)
  # Evaluate accuracy on validation data
  performance4 = cnn_model4.evaluate(valid_X, to_categorical(valid_y), verbose=0)

  print("Hyperparameters: ", hyperparameter, "Accuracy: ", performance4[1])
  print("----------------------------------------------------")
  # We want to minimize loss i.e. negative of accuracy
  return({"status": STATUS_OK, "loss": -1*performance4[1], "model":cnn_model4})
  

# Define search space for hyper-parameters
space = {
    # The kernel_size for convolutions:
    'conv_kernel_size': hp.choice('conv_kernel_size', [1, 3, 5]),
    # Uniform distribution in finding appropriate dropout values
    'dropout_prob': hp.uniform('dropout_prob', 0.1, 0.35),
    # Choice of optimizer 
    'optimizer': hp.choice('optimizer', ['Adam', 'sgd']),
}

trials = Trials()

# Find the best hyperparameters
best = fmin(
        optimize_cnn,
        space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=25,
    )

print("==================================")
print("Best Hyperparameters", best)


test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

performance5 = test_model.evaluate(test_images_3d, to_categorical(test_labels))

print("==================================")
print("Test Accuracy: ", performance5[1])

Hyperparameters: 
{'conv_kernel_size': 5, 'dropout_prob': 0.28377801403041947, 'optimizer': 'Adam'}
Accuracy: 
0.508463978767395
----------------------------------------------------
Hyperparameters: 
{'conv_kernel_size': 1, 'dropout_prob': 0.14430334361457262, 'optimizer': 'sgd'}
Accuracy: 
0.25726228952407837
----------------------------------------------------
Hyperparameters: 
{'conv_kernel_size': 1, 'dropout_prob': 0.32484540672378465, 'optimizer': 'sgd'}
Accuracy: 
0.2601880729198456
----------------------------------------------------
Hyperparameters: 
{'conv_kernel_size': 5, 'dropout_prob': 0.14020714080902222, 'optimizer': 'Adam'}
Accuracy: 
0.47398120164871216
----------------------------------------------------
Hyperparameters: 
{'conv_kernel_size': 3, 'dropout_prob': 0.1504675081066891, 'optimizer': 'sgd'}
Accuracy: 
0.25997909903526306
----------------------------------------------------
Hyperparameters: 
{'conv_kernel_size': 5, 'dropout_prob': 0.25124877102255794, 'optimiz